# PyTorch - homework 1


Please run the whole notebook with your code and submit the `.ipynb` file that includes your answers. 

In [1]:
from termcolor import colored

student_number="1004657"
student_name="Samuel Sim Wei Xuan"

print(colored("Homework by "  + student_name + ', number: ' + student_number,'red'))

Homework by Samuel Sim Wei Xuan, number: 1004657


 ## Question 1 -- matrix multiplication

Implement the following mathematical operation on both the CPU and GPU (use Google Colab or another cloud service if you don't have a GPU in your computer). Print:

a) which type of GPU card you have and 

b) show the computation time for both CPU and GPU (using PyTorch). 

c) How much % fast is the GPU? 

 The operation to implement is the dot product $C = B * A^T$

 whereby $A$ is a random matrix of size $20,000 \times 1,000$ and $B$ is a random matrix of size $2,000 \times 1,000$. In addition to the required information asked above:
 
 d) please also print the resulting two $C$ matrices (they should be the same btw). 
 



In [7]:
# implement solution here

# a) which type of GPU card you have and
!nvidia-smi

# b) show the computation time for both CPU and GPU (using PyTorch).
import time
import torch

A = torch.rand(20000, 1000)
A_T = torch.transpose(A,0,1)
B = torch.rand(2000, 1000)

cpu_start_time = time.time()
cpu_result = torch.mm(B, A_T)
cpu_end_time = time.time()
cpu_time_elapsed = cpu_end_time - cpu_start_time
print('CPU computational time = {}s.'.format(cpu_time_elapsed))

gpu_start_time = time.time()
gpu_result = torch.mm(B.cuda(), A_T.cuda())
gpu_end_time = time.time()
gpu_time_elapsed = gpu_end_time - gpu_start_time
print('GPU computational time = {}s.'.format(gpu_time_elapsed))

# c) How much % fast is the GPU?
print("The GPU is {}% faster than the CPU.".format(round((cpu_time_elapsed-gpu_time_elapsed)/cpu_time_elapsed*100),2))

# d) please also print the resulting two  C  matrices (they should be the same btw).
print(cpu_result)
print(gpu_result)


Mon Jun 20 21:05:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 511.79       Driver Version: 511.79       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8    N/A /  N/A |   1033MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Question 2 - grad


Find the gradient (partial derivatives) of the function $g(w)$ below. 

Let  $w=[w_1,w_2]^T$

Consider  $g(w)=2w_1w_2+w_2cos(w_1)$

a) In PyTorch, compute:   $\nabla g(w)$ 

 and verify that $\nabla g([\pi,1])=[2,2\pi−1]^T$ using the grad function, whereby the first position is the partial for $w_1$ and the second position is the partial for $w_2$. 

b) You can also write a function to manually calculate these partial derivatives! You can review your differential equations math at [here](https://www.wolframalpha.com/input/?i=derivative+y+cos%28x%29) and implement this as a second function below to verify that it comes to the same solution. 


In [3]:
# write your solution here
import math

# a) In PyTorch, compute del g(w)
def g(w): 
    return 2*w[0]*w[1] + w[1]*torch.cos(w[0])

input_w = torch.tensor([math.pi, 1.0], requires_grad=True)
output_g = g(input_w)
output_g.backward()

print('PyTorch Gradient of g(w) = {}'.format([input_w.grad[0].item(),input_w.grad[1].item()]))

# b) Manual calculation of the gradient of g(w)
def partial_derivatives(w): 
    return [2*w[1] - w[1]*torch.sin(w[0]), 2*w[0] + torch.cos(w[0])]

derivatives = partial_derivatives(input_w)

print('Manual Gradient of g(w) = {}'.format([derivatives[0].item(), derivatives[1].item()]))

PyTorch Gradient of g(w) = [2.0, 5.2831854820251465]
Manual Gradient of g(w) = [2.0, 5.2831854820251465]


## Question 3 - dance hit song prediction

Implement logistic regression in PyTorch for the following dance hit song prediction training dataset: 
https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv

 * Input variables: a number of audio features (most already standardized so don't worry about that)
 * Target variable: Topclass1030: 
   * 1 means it was a top 10 hit song; 
   * 0 means it never went above top 30 position.

This dataset is derived from my paper on dance hit song prediction, for full description of features have a look at https://arxiv.org/abs/1905.08076. 

Print the evolution of the loss every few epochs and train the model until it converges. 
 
 After training the logistic regression model, calculate the prediction accuracy on the test set: 
 https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv








In [4]:
# Your code here
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

# load data
dataset = pd.read_csv('https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030training.csv')
output = dataset['Topclass1030']
input = dataset.drop('Topclass1030', axis=1)

# define logistic regression model
class Logistic_Regression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Logistic_Regression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self,x):
        x = self.linear(x)
        x = torch.sigmoid(x)
        return x

model = Logistic_Regression(input.shape[1], 1)
# if torch.cuda.is_available():
#     model = model.cuda()

loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

# train model
num_epochs = 2000
num_steps = input.shape[0]

for epoch in range(num_epochs):
    for step in range(num_steps):
        batch_x = torch.tensor(input.loc[step].values).float()
        batch_y = torch.tensor([output.loc[step]]).float()

        # if torch.cuda.is_available():
        #     batch_x = batch_x.cuda()
        #     batch_y = batch_y.cuda()

        optimizer.zero_grad() 
        y_hat = model(batch_x) 

        loss = loss_function(y_hat, batch_y) 
        loss.backward() 
        optimizer.step() 
    
    if epoch % 100 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, num_epochs, loss.item()))

Epoch [0/2000], Loss: 0.6791
Epoch [100/2000], Loss: 0.5735
Epoch [200/2000], Loss: 0.5562
Epoch [300/2000], Loss: 0.5280
Epoch [400/2000], Loss: 0.5051
Epoch [500/2000], Loss: 0.4885
Epoch [600/2000], Loss: 0.4765
Epoch [700/2000], Loss: 0.4679
Epoch [800/2000], Loss: 0.4617
Epoch [900/2000], Loss: 0.4571
Epoch [1000/2000], Loss: 0.4538
Epoch [1100/2000], Loss: 0.4514
Epoch [1200/2000], Loss: 0.4498
Epoch [1300/2000], Loss: 0.4487
Epoch [1400/2000], Loss: 0.4481
Epoch [1500/2000], Loss: 0.4478
Epoch [1600/2000], Loss: 0.4479
Epoch [1700/2000], Loss: 0.4482
Epoch [1800/2000], Loss: 0.4487
Epoch [1900/2000], Loss: 0.4493


Run the below code to test the accuracy of your model on the training set: 

In [5]:
import pandas as pd 

test = pd.read_csv('https://dorax.s3.ap-south-1.amazonaws.com/herremans_hit_1030test.csv')
labels = test.iloc[:,-1]
test = test.drop('Topclass1030', axis=1)
testdata = torch.Tensor(test.values)
testlabels = torch.Tensor(labels.values).view(-1,1)

TP = 0
TN = 0
FN = 0
FP = 0

for i in range(0, testdata.size()[0]): 
  # print(testdata[i].size())
  Xtest = torch.Tensor(testdata[i])
  y_hat = model(Xtest)
  
  if y_hat > 0.5:
    prediction = 1
  else: 
    prediction = 0

  if (prediction == testlabels[i]):
    if (prediction == 1):
      TP += 1
    else: 
      TN += 1

  else:
    if (prediction == 1):
      FP += 1
    else: 
      FN += 1

print("True Positives: {0}, True Negatives: {1}".format(TP, TN))
print("False Positives: {0}, False Negatives: {1}".format(FP, FN))
rate = TP/(FN+TP)
print("Class specific accuracy of correctly predicting a hit song is {0}".format(rate))

True Positives: 43, True Negatives: 17
False Positives: 12, False Negatives: 7
Class specific accuracy of correctly predicting a hit song is 0.86
